In [1]:
import bw2data as bd

14:09:37+0100 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [2]:
project_name = "manure"

if project_name in bd.projects:
    bd.projects.delete_project(name=project_name, delete_dir=True)

if "ecoinvent-3.11-cutoff-bw25" in bd.projects:
    bd.projects.set_current("ecoinvent-3.11-cutoff-bw25")
    bd.projects.copy_project("manure", switch=True)
    if "lci-pass_cars" in bd.databases:
        del bd.databases["lci-pass_cars"]  # Redundant db from ecoinvent-3.11-cutoff-bw25
else:
    print("The project 'ecoinvent-3.11-cutoff-bw25' is not found.")

print("active project:", bd.projects.current)

active project: manure


In [3]:
db_name = "manure_foreground"

# To avoid mistakes, we recreate the manure_foreground database from scratch.
if db_name in bd.databases:
    del bd.databases[db_name]    
db = bd.Database(db_name)
db.register()
bd.databases

Databases dictionary with 3 object(s):
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff
	manure_foreground

In [4]:
ecoinvent = bd.Database("ecoinvent-3.11-cutoff")
biosphere = bd.Database("ecoinvent-3.11-biosphere")

In [5]:
ei_elec = ecoinvent.get("6bcbd7fc891c8b3bf0dd96027c1f9b0e")
print(f"Selected ecoinvent node: {ei_elec['name']} | {ei_elec['location']}")

Selected ecoinvent node: market for electricity, medium voltage | DK


In [6]:
ei_methane = biosphere.get("da1157e2-7593-4dfd-80dd-a3449b37a4d8")
print(f"Selected biosphere node: {ei_methane['name']} | {ei_methane['categories']}")

Selected biosphere node: Methane, non-fossil | ('air',)


Creating the nodes of the foreground database

In [7]:
manure_production = db.new_node(
    code="manure_production",
    name='manure_production',
    location="DK",
    type=bd.labels.process_node_default,
)
manure_production.save()

manure = db.new_node(
    code="manure",
    name='manure',
    unit="kg",
    type=bd.labels.product_node_default,
)
manure.save()

dewatering = db.new_node(
    code="dewatering",
    name='dewatering',
    location='DK',
    type=bd.labels.process_node_default,
)
dewatering.save()
 
dewatering_product = db.new_node(
    code = "dewatering_product",
    name = 'dewatering_product',
    unit="kg",
    type=bd.labels.product_node_default,
) 
dewatering_product.save()

soil_application = db.new_node(
    code = "soil_application",
    name = 'soil_application',
    unit="kg",
    type=bd.labels.process_node_default,
) 
soil_application.save()

storage = db.new_node(
    code = "storage",
    name = 'storage',
    unit="kg",
    type=bd.labels.process_node_default,
) 
storage.save()

soil_application_product = db.new_node(
    code = "soil_application_product",
    name = 'soil_application_product',
    unit="kg",
    type=bd.labels.product_node_default,
) 
soil_application_product.save()

storage_product = db.new_node(
    code = "storage_product",
    name = 'storage_product',
    unit="kg",
    type=bd.labels.product_node_default,
) 
storage_product.save()

manure_mgmt = db.new_node(
    code = "manure_mgmt",
    name = 'manure_mgmt',
    unit="kg",
    type=bd.labels.process_node_default,
) 
manure_mgmt.save()

manure_managed = db.new_node(
    code = "manure_managed",
    name = 'manure_managed',
    unit="kg",
    type=bd.labels.product_node_default,
) 
manure_managed.save()

In [8]:
manure_production.new_edge(
    amount=1,
    location= "DK",
    name="manure_1",
    unit = "kg",
    input=manure,
    type=bd.labels.production_edge_default,
).save()

dewatering.new_edge(
    amount=1,
    location= "DK",
    name="manure_2",
    unit = "kg",
    input=manure,
    type=bd.labels.consumption_edge_default,
).save()

dewatering.new_edge(
    amount=1,
    location= "DK",
    name="dprod_1",
    unit = "kg",
    input=dewatering_product,
    type=bd.labels.production_edge_default,
).save()

dewatering.new_edge(
    name = "ei_elect",
    location = "DK", 
    amount =1,
    unit = "kwh",
    input=ei_elec, #calling directly the selected ecoinvent process
    type=bd.labels.consumption_edge_default,
).save()

soil_application.new_edge(
    name= "fibres_1",
    amount=1,
    unit = "kg",
    location= "DK",
    input=dewatering_product,
    type=bd.labels.consumption_edge_default,
).save()

soil_application.new_edge(
    name="fibres_2",
    amount=1,
    unit="kg",
    location="DK",
    input=soil_application_product,
    type=bd.labels.production_edge_default,
).save()

storage.new_edge(
    name= "liquid_1",
    amount=1,
    unit = "kg",
    location= "DK",
    input=dewatering_product,
    type=bd.labels.consumption_edge_default,
).save()

storage.new_edge(
    name= "liquid_2",
    amount=1,
    unit = "kg",
    location= "DK",
    input=storage_product,
    type=bd.labels.production_edge_default,
).save()

storage.new_edge(
    name = "methane",
    location = "DK",
    amount=1,
    unit="kg",
    input=ei_methane,
    type=bd.labels.biosphere_edge_default,
).save()

manure_mgmt.new_edge(
    name= "soil_prod",
    amount=1,
    location= "DK",
    input=soil_application_product,
    type=bd.labels.consumption_edge_default,
).save()

manure_mgmt.new_edge(
    name= "storage_prod",
    amount=1,
    location= "DK",
    input=storage_product,
    type=bd.labels.consumption_edge_default,
).save()

manure_mgmt.new_edge(
    name= "managed_manure",
    amount=1,
    location= "DK",
    input=manure_managed,
    type=bd.labels.production_edge_default,
).save()

In [9]:
for act in bd.Database(db_name):
    print()
    print(act)
    for e in act.edges():
        print("  ", e)


'manure' (kg, GLO, None)

'manure_managed' (kg, GLO, None)

'manure_mgmt' (kg, GLO, None)
   Exchange: 1 kg 'soil_application_product' (kg, GLO, None) to 'manure_mgmt' (kg, GLO, None)
   Exchange: 1 kg 'storage_product' (kg, GLO, None) to 'manure_mgmt' (kg, GLO, None)
   Exchange: 1 kg 'manure_managed' (kg, GLO, None) from 'manure_mgmt' (kg, GLO, None)

'manure_production' (None, DK, None)
   Exchange: 1 kg 'manure' (kg, GLO, None) from 'manure_production' (None, DK, None)

'soil_application' (kg, GLO, None)
   Exchange: 1 kg 'dewatering_product' (kg, GLO, None) to 'soil_application' (kg, GLO, None)
   Exchange: 1 kg 'soil_application_product' (kg, GLO, None) from 'soil_application' (kg, GLO, None)

'storage' (kg, GLO, None)
   Exchange: 1 kg 'dewatering_product' (kg, GLO, None) to 'storage' (kg, GLO, None)
   Exchange: 1 kg 'storage_product' (kg, GLO, None) from 'storage' (kg, GLO, None)
   Exchange: 1 kilogram 'Methane, non-fossil' (kilogram, None, ('air',)) to 'storage' (kg, GLO, N

In [10]:
[e for e in storage.biosphere()][0]["input"]

('ecoinvent-3.11-biosphere', 'da1157e2-7593-4dfd-80dd-a3449b37a4d8')

In [11]:
from IPython.display import Markdown

def convert_db_to_mermaid(database_name, max_nodes=50, include_biosphere=True):
    
    mermaid_string = "flowchart LR"  # This will become the output to be copied/called in mermaid

    if database_name not in bd.databases:
        raise Exception("Database not found.") 
    else:
        db = bd.Database(database_name)

    if len(db) > max_nodes:
        raise Exception(f"Database has too many nodes: {len(db)}. Current threshold: {max_nodes}. Adjust max_nodes argument.")
    
    for node in db:
        
        
        # Production exchanges
        for edge in [edge for edge in node.production()]:
            process_name = node['name'].replace(" ", "_")
            product_db = edge["input"][0]
            product_code = edge["input"][1]
            product_name = bd.get_node(database=product_db, code=product_code)["name"].replace(" ", "_")
            if product_db != database_name:
                product_name = product_name + "_" + product_db 
            mermaid_string += "\n" + f"\t{process_name}(({process_name})) --> {product_name}{{{product_name}}}"
    
        # Consumption exchanges        
        for edge in [edge for edge in node.technosphere()]:
            if edge["type"] != "production":
                process_name = node['name'].replace(" ", "_")
                product_db = edge["input"][0]
                product_code = edge["input"][1]
                product_name = bd.get_node(database=product_db, code=product_code)["name"].replace(" ", "_")
                if product_db != database_name:
                    product_name = product_name + "_" + product_db
                mermaid_string += "\n" + f"{product_name}{{{product_name}}} --> \t{process_name}"
    
        # Biosphere exchanges 
        if include_biosphere:
            for edge in node.biosphere():
                process_name = node['name'].replace(" ", "_")
                product_db = edge["input"][0]
                product_code = edge["input"][1]
                biosphere_name = bd.get_node(database=product_db, code=product_code)["name"].replace(" ", "_")
                if product_db != database_name:
                    biosphere_name = biosphere_name + "_" + product_db
                mermaid_string += "\n" + f"\t{process_name} --> {biosphere_name}[{biosphere_name}]"
    
    return mermaid_string

In [12]:
mermaid_string = convert_db_to_mermaid(db_name)
# mermaid_string = convert_db_to_mermaid(db_name, max_nodes=5)

Markdown(f"```mermaid\n{mermaid_string}\n```")

```mermaid
flowchart LR
	soil_application((soil_application)) --> soil_application_product{soil_application_product}
dewatering_product{dewatering_product} --> 	soil_application
	dewatering((dewatering)) --> dewatering_product{dewatering_product}
manure{manure} --> 	dewatering
market_for_electricity,_medium_voltage_ecoinvent-3.11-cutoff{market_for_electricity,_medium_voltage_ecoinvent-3.11-cutoff} --> 	dewatering
	manure_mgmt((manure_mgmt)) --> manure_managed{manure_managed}
soil_application_product{soil_application_product} --> 	manure_mgmt
storage_product{storage_product} --> 	manure_mgmt
	storage((storage)) --> storage_product{storage_product}
dewatering_product{dewatering_product} --> 	storage
	storage --> Methane,_non-fossil_ecoinvent-3.11-biosphere[Methane,_non-fossil_ecoinvent-3.11-biosphere]
	manure_production((manure_production)) --> manure{manure}
```